In [1]:
from dask.distributed import Client, progress
# HPC
# client = Client(scheduler_file='/glade/scratch/jhamman/scheduler.json')
# client

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

/srv/conda/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/srv/conda/lib/python3.6/site-packages/dask_kubernetes/config.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
client = Client(cluster)
client

Client Scheduler: tcp://10.32.2.116:40897 Dashboard: /user/0000-0001-7783-5629/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
import sys, glob
import numpy as np
import xarray as xr
import xscale.spectral.fft as xfft
import Wavenum_freq_spec_func as wfs


In [4]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
from matplotlib.colors import LogNorm

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic


In [ ]:
import gcsfs

fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='anon', access='read_only')

gcsmapgrid=gcsfs.mapping.GCSMap('pangeo-data/llc4320_surface/grid',gcs=fs,check=False,create=False)
dsgrid=xr.open_zarr(gcsmapgrid)


In [ ]:

gcsmap=gcsfs.mapping.GCSMap('pangeo-data/llc4320_surface/U',gcs=fs,check=False,create=False)
dsu=xr.open_zarr(gcsmap)

In [ ]:

gcsmap=gcsfs.mapping.GCSMap('pangeo-data/llc4320_surface/V',gcs=fs,check=False,create=False)
dsv=xr.open_zarr(gcsmap)

In [ ]:
lat=dsgrid.sel(face=2)['YC']
lon=dsgrid.sel(face=2)['XC']


In [ ]:
latmin = 40.0; latmax = 45.0;
lonmin = -40.0; lonmax = -35.0;


In [ ]:
latbox=lat.where((lonmin<lon) & (lon<lonmax) & (latmin<lat) & (lat<latmax),drop=True)
lonbox=lon.where((lonmin<lon) & (lon<lonmax) & (latmin<lat) & (lat<latmax),drop=True)

In [ ]:
u_JFM=dsu.sel(face=2,time=slice('2012-01-01','2012-03-31'))['U']
v_JFM=dsv.sel(face=2,time=slice('2012-01-01','2012-03-31'))['V']

u_JFM_i=u_JFM.rename({'i_g':'i'})
v_JFM_j=v_JFM.rename({'j_g':'j'})

u_JFM_box=u_JFM_i.where((lonmin<lon) & (lon<lonmax) & (latmin<lat) & (lat<latmax),drop=True)
v_JFM_box=v_JFM_j.where((lonmin<lon) & (lon<lonmax) & (latmin<lat) & (lat<latmax),drop=True)

# - remove NaN
u = u_JFM_box.interpolate_na(dim='j')
v = v_JFM_box.interpolate_na(dim='j')

# - get dx and dy
dx,dy = wfs.get_dx_dy(u[0],lonbox,latbox)


#... Detrend data in all dimension ...
print('Detrend data in all dimension')
u = wfs.detrendn(u,axes=[0,1,2])
v = wfs.detrendn(v,axes=[0,1,2])

#... Apply hanning windowing ...') 
print('Apply hanning windowing')
u = wfs.apply_window(u, u.dims, window_type='hanning')
v = wfs.apply_window(v, v.dims, window_type='hanning')

uhat = xfft.fft(u, dim=('time', 'i', 'j'), dx={'i': dx, 'j': dx}, sym=True)
vhat = xfft.fft(v, dim=('time', 'i', 'j'), dx={'i': dx, 'j': dx}, sym=True)

u_psd = xfft.psd(uhat)
v_psd = xfft.psd(vhat)

frequency = uhat.f_time
kx = uhat.f_i
ky = uhat.f_j

wavenumber,kradial = wfs.get_wavnum_kradial(kx,ky)

#... Get numpy array ... 
print('Get numpy array')
u_psd_np = u_psd.values
v_psd_np = v_psd.values

u_wavenum_freq_spectrum = wfs.get_f_k_in_2D(kradial,wavenumber,u_psd_np)
v_wavenum_freq_spectrum = wfs.get_f_k_in_2D(kradial,wavenumber,v_psd_np)

KE_JFM_wavenum_freq_spectrum=0.5*(u_wavenum_freq_spectrum+v_wavenum_freq_spectrum)


In [ ]:
u_JAS=dsu.sel(face=2,time=slice('2012-06-01','2012-09-23'))['U']
v_JAS=dsv.sel(face=2,time=slice('2012-06-01','2012-09-23'))['V']

u_JAS_i=u_JAS.rename({'i_g':'i'})
v_JAS_j=v_JAS.rename({'j_g':'j'})

u_JAS_box=u_JAS_i.where((lonmin<lon) & (lon<lonmax) & (latmin<lat) & (lat<latmax),drop=True)
v_JAS_box=v_JAS_j.where((lonmin<lon) & (lon<lonmax) & (latmin<lat) & (lat<latmax),drop=True)

# - remove NaN
u = u_JAS_box.interpolate_na(dim='j')
v = v_JAS_box.interpolate_na(dim='j')

# - get dx and dy
dx,dy = wfs.get_dx_dy(u[0],lonbox,latbox)


#... Detrend data in all dimension ...
print('Detrend data in all dimension')
u = wfs.detrendn(u,axes=[0,1,2])
v = wfs.detrendn(v,axes=[0,1,2])

#... Apply hanning windowing ...') 
print('Apply hanning windowing')
u = wfs.apply_window(u, u.dims, window_type='hanning')
v = wfs.apply_window(v, v.dims, window_type='hanning')

uhat = xfft.fft(u, dim=('time', 'i', 'j'), dx={'i': dx, 'j': dx}, sym=True)
vhat = xfft.fft(v, dim=('time', 'i', 'j'), dx={'i': dx, 'j': dx}, sym=True)

u_psd = xfft.psd(uhat)
v_psd = xfft.psd(vhat)

frequency = uhat.f_time
kx = uhat.f_i
ky = uhat.f_j

wavenumber,kradial = wfs.get_wavnum_kradial(kx,ky)

#... Get numpy array ... 
print('Get numpy array')
u_psd_np = u_psd.values
v_psd_np = v_psd.values

u_wavenum_freq_spectrum = wfs.get_f_k_in_2D(kradial,wavenumber,u_psd_np)
v_wavenum_freq_spectrum = wfs.get_f_k_in_2D(kradial,wavenumber,v_psd_np)

KE_JAS_wavenum_freq_spectrum=0.5*(u_wavenum_freq_spectrum+v_wavenum_freq_spectrum)


In [ ]:
sec_to_hour = 3600.0
norm = LogNorm(vmin=0.001,vmax=1000)
cmap = div_cmap

fig=plt.figure(figsize=(30,20))

ax = plt.subplot(121)
plt.pcolormesh(wavenumber,sec_to_hour*frequency,KE_JFM_wavenum_freq_spectrum,norm=norm,cmap=cmap)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel('wavenumber (cpkm)',fontsize=15)
ax.set_ylabel('frequency (cph)',fontsize=15)
ax.set_xlim(wavenumber.min(),wavenumber.max())
ax.set_ylim(1E-5,8E-1)
ax.set_title('KE spectrum JFM Small Box 1h',size=18)
ax.tick_params(labelsize=15)
plt.legend
plt.colorbar()

ax = plt.subplot(122)
plt.pcolormesh(wavenumber,sec_to_hour*frequency,KE_JAS_wavenum_freq_spectrum,norm=norm,cmap=cmap)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel('wavenumber (cpkm)',fontsize=15)
ax.set_ylabel('frequency (cph)',fontsize=15)
ax.set_xlim(wavenumber.min(),wavenumber.max())
ax.set_ylim(1E-5,8E-1)
ax.set_title('KE spectrum JAS Small Box 1h',size=18)
ax.tick_params(labelsize=15)
plt.legend
plt.colorbar()
